<a href="https://colab.research.google.com/github/abilincoln1/FP-Markets-AI-Trader-/blob/main/EV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open('/content/deepseek_python_20250818_540d41.py', 'r') as f:
    print(f.read())

# %% [markdown]
# # 🚛 EV Fleet Battery Failure Predictor (UK-Optimized)
# *Train an edge-compatible model to predict battery failures 24hrs in advance*

# %% [markdown]
# ## Step 1: Install Libraries
!pip install pandas scikit-learn tensorflow tflite-support obd-py pyarrow -q

# %% [markdown]
# ## Step 2: Load UK Fleet Data (Sample Dataset)
import pandas as pd
from google.colab import files

# Upload your OBD-II logs (or use our sample)
uploaded = files.upload()
df = pd.read_parquet(next(iter(uploaded.keys())))

# Sample synthetic data if no upload
if df.empty:
  print("Using synthetic UK data...")
  data = {
      'voltage': [13.2, 12.8, 14.1, 11.9, 13.5],
      'temp_c': [18, 22, -3, 35, 19],  # UK weather range
      'charge_cycles': [120, 345, 87, 500, 210],
      'failed_next_24hrs': [0, 1, 0, 1, 0]  # Target
  }
  df = pd.DataFrame(data)

# %% [markdown]
# ## Step 3: Train Lightweight Model (Edge Optimized)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_se

In [3]:
# %% [markdown]
# # 🚛 EV Fleet Battery Failure Predictor (UK-Optimized)
# *Train an edge-compatible model to predict battery failures 24hrs in advance*

# %% [markdown]
# ## Step 1: Install Libraries
# Removed obd-py as it might not be available in this environment and explore alternatives if needed.
# Also removed tflite-support as it is not needed for the conversion step using from_saved_model.
!pip install pandas scikit-learn tensorflow pyarrow -q

# %% [markdown]
# ## Step 2: Load UK Fleet Data (Sample Dataset)
import pandas as pd
from google.colab import files

# Upload your OBD-II logs (or use our sample)
# uploaded = files.upload() # Commenting out file upload for now to use synthetic data
# df = pd.read_parquet(next(iter(uploaded.keys()))) # Commenting out file upload for now to use synthetic data

# Sample synthetic data if no upload
# if df.empty: # This check will always be true after commenting out file upload
print("Using synthetic UK data...")
data = {
    'voltage': [13.2, 12.8, 14.1, 11.9, 13.5, 12.5, 13.0, 11.5, 14.0, 12.0],
    'temp_c': [18, 22, -3, 35, 19, 15, 25, 0, 30, 10],  # UK weather range
    'charge_cycles': [120, 345, 87, 500, 210, 150, 400, 50, 600, 90],
    'failed_next_24hrs': [0, 1, 0, 1, 0, 0, 1, 0, 1, 0]  # Target
}
df = pd.DataFrame(data)


# %% [markdown]
# ## Step 3: Train Lightweight Model (Edge Optimized)
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Feature engineering
df['voltage_drop'] = df['voltage'].rolling(3).std().fillna(0)
df['cold_weather_risk'] = (df['temp_c'] < 5).astype(int)

X = df[['voltage', 'temp_c', 'charge_cycles', 'voltage_drop', 'cold_weather_risk']]
y = df['failed_next_24hrs']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert pandas DataFrames to TensorFlow Tensors
X_train_tf = tf.constant(X_train.values, dtype=tf.float32)
X_test_tf = tf.constant(X_test.values, dtype=tf.float32)
y_train_tf = tf.constant(y_train.values, dtype=tf.float32)
y_test_tf = tf.constant(y_test.values, dtype=tf.float32)


# Build a simple Keras model for binary classification
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)), # Input layer matching number of features
    tf.keras.layers.Dense(16, activation='relu'), # Hidden layer with ReLU activation
    tf.keras.layers.Dropout(0.2), # Dropout for regularization
    tf.keras.layers.Dense(8, activation='relu'), # Another hidden layer
    tf.keras.layers.Dense(1, activation='sigmoid') # Output layer for binary classification with sigmoid
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
print("\nTraining the TensorFlow/Keras model...")
history = model.fit(X_train_tf, y_train_tf, epochs=100, batch_size=8, validation_split=0.2, verbose=0) # Train for 100 epochs

# Evaluate the model
loss, accuracy = model.evaluate(X_test_tf, y_test_tf, verbose=0)
print(f"TensorFlow/Keras Model Accuracy: {accuracy:.2f}")


# %% [markdown]
# ## Step 4: Convert to TensorFlow Lite (For OBD-II Dongles)
import tensorflow as tf
# from sklearn.tensorflow import export_saved_model # Removed incorrect import

# Convert sklearn → TF → TFLite
# export_saved_model(model, "saved_model") # Removed incorrect usage
# converter = tf.lite.TFLiteConverter.from_saved_model("saved_model") # This requires a SavedModel format
# A standard way to convert a trained Keras model to TFLite is shown below.
# Since we are using RandomForestClassifier from sklearn, direct conversion to TFLite is not straightforward.
# One approach is to rebuild the model in TensorFlow/Keras or use a library that supports converting sklearn models to TensorFlow format.

# Now that we have a TensorFlow/Keras model, we can convert it to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Download for edge devices
with open('battery_predictor.tflite', 'wb') as f:
  f.write(tflite_model)
files.download('battery_predictor.tflite') # Uncommented download

print("\n✅ TFLite model ready for Raspberry Pi/OBD-II!")
print("The TFLite model 'battery_predictor.tflite' has been saved.")

Using synthetic UK data...

Training the TensorFlow/Keras model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


TensorFlow/Keras Model Accuracy: 0.00
Saved artifact at '/tmp/tmpcex3h0vs'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134734954758224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134734954761104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134734954761296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134734954762640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134734954761680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134734954758800: TensorSpec(shape=(), dtype=tf.resource, name=None)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ TFLite model ready for Raspberry Pi/OBD-II!
The TFLite model 'battery_predictor.tflite' has been saved.
